#Dependencies

In [1]:
%pip install --upgrade --quiet  langchain langchain-community langchainhub langchain-openai langchain-chroma bs4

Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install pdfplumber

In [3]:
import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_chroma import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [4]:
%pip install -qU langchain-openai

Note: you may need to restart the kernel to use updated packages.


In [5]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

In [17]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

In [62]:
# Load, chunk and index the contents of the blog.
# from langchain_community.document_loaders import PDFPlumberLoader
# loader = PDFPlumberLoader("./data/machine_learning.pdf")

# for .txt:
from langchain.document_loaders import TextLoader
loader = TextLoader("./data/alice_in_wonderland.txt", encoding = 'UTF-8')
docs = loader.load()

# for XML
# from langchain_community.document_loaders import UnstructuredXMLLoader
# loader = UnstructuredXMLLoader(
#     "example_data/factbook.xml",
# )
# docs = loader.load()

# for word:
# %pip install --upgrade --quiet  docx2txt
# from langchain_community.document_loaders import Docx2txtLoader
# loader = Docx2txtLoader("example_data/fake.docx")
# docs = loader.load()



In [64]:
# Split
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 0)
all_splits = text_splitter.split_documents(docs)

In [65]:
# Store splits
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
vectorstore = Chroma.from_documents(documents=all_splits, embedding=OpenAIEmbeddings())

# RAG prompt
from langchain import hub
prompt = hub.pull("rlm/rag-prompt")
# i have taken template directly from langchainhub

# LLM
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

# RetrievalQA
# here context is vectorstore.as_retriever() 
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectorstore.as_retriever(),
    chain_type_kwargs={"prompt": prompt}
)

In [ ]:
question = "whose resume is this?"
result = qa_chain({"query": question})
result["result"]

In [24]:
print(prompt.messages)

[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"))]


In [67]:
# cleanup
vectorstore.delete_collection()

***
For real-time information we use serpapi- using api
***

In [ ]:
%pip install google-search-results

In [73]:
import getpass
import os

serpapi_api_key = os.environ["serpapi_api_key"] = getpass.getpass()

In [75]:
from langchain.agents import AgentType
from langchain.agents import load_tools 
from langchain.agents import initialize_agent
from langchain.llms import OpenAI

In [76]:
tool=load_tools(["serpapi"],serpapi_api_key=serpapi_api_key,llm=llm)

In [77]:
agent=initialize_agent(tool,llm,agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,verbose=True)

c:\Users\chait\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [81]:
agent.run("can you tell who won the cricket worldcup recently?")

c:\Users\chait\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
I should search for the latest information on the Cricket World Cup winner.
Action: Search
Action Input: "Cricket World Cup winner 2021"
Observation: {'title': "ICC Men's T20 World Cup", 'thumbnail': 'https://serpapi.com/searches/661bf3c68ccee0d216849182/images/4406afe442f1753033ac98f0764bc2d4190deb86dd83d809d8d1243095eac367.png', 'games': [{'tournament': "ICC Men's T20 World Cup", 'stadium': 'Dubai International Stadium', 'date': 'Nov 14, 21', 'teams': [{'name': 'New Zealand', 'score': '172/4 (20)', 'thumbnail': 'https://serpapi.com/searches/661bf3c68ccee0d216849182/images/4406afe442f175304ab806339654480772fdf1e42de1074f0499b0ad5a48260afa5ef4a09b1b32aeabe997973f715616.png'}, {'name': 'Australia', 'score': '173/2 (18.5)', 'thumbnail': 'https://serpapi.com/searches/661bf3c68ccee0d216849182/images/4406afe442f175304ab806339654480772fdf1e42de1074f39dae8bf90813bbe1066f5ea4cc151c1ea3fde95ec0673d6.png'}], 'status': 'AUS won by 8 wickets (7 balls left)'}

'Australia'